# Tutorial: Counterfactual explanations for scorecard with continuous target

This tutorial shows how to generate counterfactual explanations on scorecard models with continuous target. The dataset for this tutorial is https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html.

In [1]:
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import HuberRegressor

from optbinning import BinningProcess
from optbinning import Scorecard
from optbinning.scorecard import Counterfactual

Load the dataset

In [2]:
data = fetch_california_housing()

target = "target"
variable_names = data.feature_names
X = pd.DataFrame(data.data, columns=variable_names)
y = data.target

#### Scorecard model

First, we develop a scorecard following the steps presented in previous tutorials.

In [3]:
binning_process = BinningProcess(variable_names)

estimator = HuberRegressor(max_iter=200)

scorecard = Scorecard(binning_process=binning_process,
                      estimator=estimator, scaling_method="min_max",
                      scaling_method_params={"min": 0, "max": 100},
                      reverse_scorecard=True,verbose=True)

scorecard.fit(X, y)

2021-10-23 20:02:53,105 | INFO : Scorecard building process started.
2021-10-23 20:02:53,107 | INFO : Options: check parameters.
2021-10-23 20:02:53,109 | INFO : Dataset: continuous target.
2021-10-23 20:02:53,110 | INFO : Binning process started.
2021-10-23 20:02:54,929 | INFO : Binning process terminated. Time: 1.8181s
2021-10-23 20:02:54,930 | INFO : Fitting estimator.
2021-10-23 20:02:55,670 | INFO : Fitting terminated. Time 0.7399s
2021-10-23 20:02:55,672 | INFO : Scorecard table building started.
2021-10-23 20:02:55,830 | INFO : Scorecard table terminated. Time: 0.1575s
2021-10-23 20:02:55,831 | INFO : Scorecard building process terminated. Time: 2.7254s


Scorecard(binning_process=BinningProcess(variable_names=['MedInc', 'HouseAge',
                                                         'AveRooms',
                                                         'AveBedrms',
                                                         'Population',
                                                         'AveOccup', 'Latitude',
                                                         'Longitude']),
          estimator=HuberRegressor(max_iter=200), reverse_scorecard=True,
          scaling_method='min_max',
          scaling_method_params={'max': 100, 'min': 0}, verbose=True)

#### Generating counterfactual explanations

As an input data point or query, we select the first sample. Note that a query must be either a dictionary of a pandas DataFrame.

In [4]:
query = X.iloc[0, :].to_frame().T

In [5]:
query

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23


In [6]:
scorecard.predict(query)

array([4.38634097])

The predicted outcome (house value) for this query (house) is 4.38. We want to generate counterfactual explanations to find out how to increase the house value to at least 4.5.

In [7]:
cf = Counterfactual(scorecard=scorecard, verbose=True)

In [8]:
cf.fit(X)

2021-10-23 20:03:02,744 | INFO : Counterfactual fit started.
2021-10-23 20:03:02,747 | INFO : Options: check parameters.
2021-10-23 20:03:02,748 | INFO : Compute optimization problem data.
2021-10-23 20:03:02,801 | INFO : Counterfactual fit terminated. Time: 0.0575s


Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['MedInc',
                                                                                  'HouseAge',
                                                                                  'AveRooms',
                                                                                  'AveBedrms',
                                                                                  'Population',
                                                                                  'AveOccup',
                                                                                  'Latitude',
                                                                                  'Longitude']),
                                   estimator=HuberRegressor(max_iter=200),
                                   reverse_scorecard=True,
                                   scaling_method='min_max',
                                   scaling_method

In [9]:
cf.generate(query=query, y=4.5, outcome_type="continuous", n_cf=1,
            max_changes=3, hard_constraints=["min_outcome"])

2021-10-23 20:03:05,354 | INFO : Counterfactual generation started.
2021-10-23 20:03:05,357 | INFO : Options: check parameters.
2021-10-23 20:03:05,363 | INFO : Options: check objectives and constraints.
2021-10-23 20:03:05,365 | INFO : Optimizer started.
2021-10-23 20:03:05,366 | INFO : Optimizer: build model...
2021-10-23 20:03:05,452 | INFO : Optimizer: solve...
2021-10-23 20:03:05,651 | INFO : Optimizer terminated. Time: 0.2854s
2021-10-23 20:03:05,653 | INFO : Post-processing started.
2021-10-23 20:03:05,666 | INFO : Post-processing terminated. Time: 0.0118s
2021-10-23 20:03:05,668 | INFO : Counterfactual generation terminated. Status: OPTIMAL. Time: 0.3137s


Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['MedInc',
                                                                                  'HouseAge',
                                                                                  'AveRooms',
                                                                                  'AveBedrms',
                                                                                  'Population',
                                                                                  'AveOccup',
                                                                                  'Latitude',
                                                                                  'Longitude']),
                                   estimator=HuberRegressor(max_iter=200),
                                   reverse_scorecard=True,
                                   scaling_method='min_max',
                                   scaling_method

In [10]:
cf.information()

optbinning (Version 0.13.0)
Copyright (c) 2019-2021 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   42
    Number of constraints                120
    Objective value                   7.6866
    Best objective bound              7.6866

  Objectives
    proximity                         0.7069
    closeness                         6.9798

  Timing
    Total time                          0.35 sec
    Fit                                 0.06 sec   ( 16.22%)
    Solver                              0.29 sec   ( 80.46%)
    Post-processing                     0.01 sec   (  4.13%)



The generate counterfactual suggest increasing the block population, reduce the average house occupancy and change the house block longitude. None of them seems doable.

In [11]:
cf.display(show_only_changes=True, show_outcome=True)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,-,-,-,-,"[986.50, 1160.50)","(-inf, 1.95)",-,"[-122.16, -118.33)",4.520902


Now, let's generate several counterfactuals aiming to limit the house value to 4.0.

In [12]:
cf.generate(query=query, y=4.0, outcome_type="continuous", n_cf=3,
            max_changes=3,
            hard_constraints=["diversity_features", "max_outcome"],
            time_limit=30
           ).display(show_only_changes=True, show_outcome=True)

2021-10-23 20:03:24,534 | INFO : Counterfactual generation started.
2021-10-23 20:03:24,535 | INFO : Options: check parameters.
2021-10-23 20:03:24,546 | INFO : Options: check objectives and constraints.
2021-10-23 20:03:24,548 | INFO : Optimizer started.
2021-10-23 20:03:24,548 | INFO : Optimizer: build model...
2021-10-23 20:03:24,701 | INFO : Optimizer: solve...
2021-10-23 20:03:44,240 | INFO : Optimizer terminated. Time: 19.6911s
2021-10-23 20:03:44,241 | INFO : Post-processing started.
2021-10-23 20:03:44,276 | INFO : Post-processing terminated. Time: 0.0334s
2021-10-23 20:03:44,277 | INFO : Counterfactual generation terminated. Status: OPTIMAL. Time: 19.7429s


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,-,-,-,-,"[986.50, 1160.50)","[3.11, 3.24)",-,"[-122.16, -118.33)",3.968866
0,"[5.79, 6.82)",-,-,"(-inf, 0.95)",-,-,-,"[-122.16, -118.33)",3.227336
0,"[5.79, 6.82)",-,-,-,"[986.50, 1160.50)",-,-,"[-122.16, -118.33)",3.158199


And the same generation enforcing diversity on feature values.

In [13]:
cf.generate(query=query, y=3.0, outcome_type="continuous", n_cf=3,
            max_changes=3,
            hard_constraints=["diversity_features", "diversity_values", "max_outcome"],
            time_limit=30
           ).display(show_only_changes=True, show_outcome=True)

2021-10-23 20:03:51,189 | INFO : Counterfactual generation started.
2021-10-23 20:03:51,191 | INFO : Options: check parameters.
2021-10-23 20:03:51,198 | INFO : Options: check objectives and constraints.
2021-10-23 20:03:51,199 | INFO : Optimizer started.
2021-10-23 20:03:51,201 | INFO : Optimizer: build model...
2021-10-23 20:03:51,377 | INFO : Optimizer: solve...
2021-10-23 20:04:15,261 | INFO : Optimizer terminated. Time: 24.0606s
2021-10-23 20:04:15,262 | INFO : Post-processing started.
2021-10-23 20:04:15,297 | INFO : Post-processing terminated. Time: 0.0341s
2021-10-23 20:04:15,299 | INFO : Counterfactual generation terminated. Status: OPTIMAL. Time: 24.1091s


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,"[5.79, 6.82)",-,-,-,-,"[3.11, 3.24)",-,"[-118.16, inf)",2.981844
0,"[4.53, 5.04)",-,"[6.12, 6.37)",-,-,-,-,"[-118.33, -118.26)",2.486897
0,"[5.04, 5.79)",-,-,-,"[986.50, 1160.50)",-,-,"[-122.16, -118.33)",2.646881
